In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import cv2

# ==============================
# SETTINGS
# ==============================
DATASET_PATH = "/kaggle/input/datasets/nodoubttome/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration"

IMG_SIZE = 300
BATCH_SIZE = 16
EPOCHS_HEAD = 15
EPOCHS_FINE = 10
NUM_CLASSES = 9

# ==============================
# FOCAL LOSS
# ==============================
def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1. - 1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        return tf.reduce_sum(weight * cross_entropy, axis=1)
    return loss

# ==============================
# LOAD ALL IMAGE PATHS
# ==============================
image_paths = []
labels = []

for split in ["Train", "Test"]:
    split_path = os.path.join(DATASET_PATH, split)
    for class_name in os.listdir(split_path):
        class_path = os.path.join(split_path, class_name)
        for img in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img))
            labels.append(class_name)

image_paths = np.array(image_paths)
labels = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

# ==============================
# TRAINING LOOP
# ==============================
for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels_encoded)):

    print(f"\n🔥 Training Fold {fold+1}")

    train_paths, val_paths = image_paths[train_idx], image_paths[val_idx]
    train_labels, val_labels = labels_encoded[train_idx], labels_encoded[val_idx]

    def load_data(paths, labels):
        images = []
        for path in paths:
            img = cv2.imread(path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
        images = np.array(images)
        labels = keras.utils.to_categorical(labels, NUM_CLASSES)
        return images, labels

    X_train, y_train = load_data(train_paths, train_labels)
    X_val, y_val = load_data(val_paths, val_labels)

    # ------------------ MODEL ------------------
    base_model = EfficientNetB3(
        weights="imagenet",
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = preprocess_input(inputs)
    x = layers.RandomFlip("horizontal")(x)
    x = layers.RandomRotation(0.1)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss=focal_loss(),
        metrics=["accuracy"]
    )

    # ------------------ TRAIN HEAD ------------------
    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS_HEAD,
        batch_size=BATCH_SIZE,
        verbose=1
    )

    # ------------------ FINE TUNE LAST 40 LAYERS ------------------
    for layer in base_model.layers[-40:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    model.compile(
        optimizer=keras.optimizers.Adam(5e-6),
        loss=focal_loss(),
        metrics=["accuracy"]
    )

    model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS_FINE,
        batch_size=BATCH_SIZE,
        verbose=1
    )

    loss, acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")

    fold_accuracies.append(acc)

    model.save(f"final_fold_{fold+1}.keras")

# ==============================
# FINAL RESULT
# ==============================
print("\n📊 Fold Accuracies:", fold_accuracies)
print("🔥 Average Accuracy:", np.mean(fold_accuracies))
